In [22]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [23]:
CLIENT_ID = 'M5H0AVJNE1ST4L1WAEHMEDYLG3UATUJIF4KAJVYKDPT0WCCQ' # your Foursquare ID
CLIENT_SECRET = 'I3GN4A3DA4NEH14FMZ3OE4DXIMRHAPDUVZH55VIOBHZRADZO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: M5H0AVJNE1ST4L1WAEHMEDYLG3UATUJIF4KAJVYKDPT0WCCQ


In [24]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["New York", 'Chicago', 'San Francisco', 'Jersey City', 'Boston']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") # Briyani place category id
    results[city] = requests.get(url).json()

In [25]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [26]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Briyani places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of Briyani places in New York =  276
Showing Top 100
Total number of Briyani places in Chicago =  216
Showing Top 100
Total number of Briyani places in San Francisco =  166
Showing Top 100
Total number of Briyani places in Jersey City =  129
Showing Top 100
Total number of Briyani places in Boston =  186
Showing Top 100


In [27]:
maps[cities[0]]

In [28]:
maps[cities[1]]

In [29]:
maps[cities[2]]

In [30]:
maps[cities[3]]

In [31]:
maps[cities[4]]

In [32]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York
Mean Distance from Mean coordinates
0.023355546704064742
Chicago
Mean Distance from Mean coordinates
0.06138547931104336
San Francisco
Mean Distance from Mean coordinates
0.029080947686810656
Jersey City
Mean Distance from Mean coordinates
0.01968274524398889
Boston
Mean Distance from Mean coordinates
0.0372898458000339


In [33]:
maps[cities[0]]

In [34]:
maps[cities[1]]

In [35]:
maps[cities[2]]

In [36]:
maps[cities[3]]

In [37]:
maps[cities[4]]

In [38]:
city = 'Jersey City'
venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 

print(city)
print("Mean Distance from Mean coordinates")
dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)
dists.sort()
print(np.mean(dists[:-1]))# Ignore the biggest distance

Jersey City
Mean Distance from Mean coordinates
0.019292222941833935
